# TreeIndex 100 文档测试自定义分块 - 128

设置很小的分块，比如 `128`，是否能提高性能。

耗时： `root<<leaf_embedding<leaf`

没有比较 `all_leaf`，因为太慢了。

创建索引耗时增加，从默认 `1024` 的 `17m46s` 增加到 `2h41m56s`，相差大致10倍，存在线性关系，即分块越多，越耗时。

建议：

- 使用 `root` 方式，但要注意可能出现错误的可能
- 尽量不要使用 `all_leaf` 方式，太慢了

## 全局设置

In [1]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=OpenAILike(
    # model="qwen2", 
    model="glm4", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 3.41 s, sys: 470 ms, total: 3.88 s
Wall time: 3.99 s


## 加载数据

In [2]:
%%time

from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader('./data').load_data()
len(documents)

CPU times: user 24.2 ms, sys: 7.91 ms, total: 32.1 ms
Wall time: 52.9 ms


99

## 创建索引

In [3]:
%%time

from llama_index.core import TreeIndex
from llama_index.core.node_parser import SentenceSplitter

index = TreeIndex.from_documents(
    documents,
    # num_children=5,
    transformations=[SentenceSplitter(chunk_size=128,chunk_overlap=10)]
)

CPU times: user 17.1 s, sys: 321 ms, total: 17.4 s
Wall time: 2h 41min 56s


In [4]:
len(index.docstore.docs.items())

11963

## 查询

### ROOT

耗时很稳定，很快。

仍然有错误，`颐和园，又称圆明园`


In [5]:
%%time

from llama_index.core.indices.tree.base import TreeRetrieverMode

query_engine = index.as_query_engine(
    retriever_mode=TreeRetrieverMode.ROOT
)

response = query_engine.query("介绍下颐和园的历史")
print(response.response)

颐和园，又称圆明园，是位于北京的一个著名的文化遗产地。它是一座融合了自然美景与人工建筑的园林，被誉为“皇家园林之最”。颐和园的历史可以追溯到清朝乾隆年间，当时它是作为皇帝的夏宫而建造的。

这座园林在历史上经历了多次重建和扩建。最初，它是由乾隆帝下令建造的，名为“清漪园”，主要用于避暑和休闲。后来，在咸丰、同治年间，由于战乱等原因，园林遭到了严重破坏。

直到光绪年间，慈禧太后重新修复了园林，并对其进行了大规模的改造，将其改名为“颐和园”。在此期间，园林的建筑风格更加注重中西合璧，增添了许多新的景观和建筑，如长廊、十七孔桥等。

颐和园不仅是中国古代皇家园林艺术的杰出代表，也是世界文化遗产之一。它以其独特的园林设计、丰富的文化内涵和精美的艺术品而闻名于世。园林内有许多重要的历史遗迹和文化遗物，包括佛香阁、昆明湖、万寿山等，都是游客参观的热点。

此外，颐和园还举办了许多文化活动，如传统音乐表演、书画展览等，成为展示中国传统文化的重要场所。
CPU times: user 8.21 ms, sys: 4.61 ms, total: 12.8 ms
Wall time: 8.5 s


### SELECT_LEAF

和默认分块大小比较，慢的不多。

从 16s 到 21s

In [6]:
%%time

query_engine = index.as_query_engine(
    retriever_mode=TreeRetrieverMode.SELECT_LEAF,
    child_branch_factor=3,
)

response = query_engine.query("介绍下颐和园的历史")
print(response.response)

颐和园位于北京，是中国著名的皇家园林之一。它原是清朝帝王的行宫和花园，始建于1750年，经过多次扩建和修缮，形成了今天我们所见的规模。

在乾隆年间，颐和园被命名为“清漪园”，后来在光绪年间改为“颐和园”。这个园林不仅是一个供皇帝及皇室成员休憩游玩的地方，也是展示皇权威严的重要场所。

园林内建筑精美，景色宜人，融合了中西方的园林艺术。其中最著名的景点包括长廊、佛香阁、十七孔桥等。在历史的长河中，颐和园见证了清朝的兴衰变迁，是中国古代园林艺术的杰出代表。
CPU times: user 116 ms, sys: 6.41 ms, total: 122 ms
Wall time: 21.7 s


### SELECT_LEAF_EMBEDDING

明显有遗漏的内容，1900年八国联军的破坏。

因为提到了1860年英法联军的破坏。

分块减少对耗时影响不大，14.9s 和 15.7s

In [8]:
%%time

query_engine = index.as_query_engine(
    retriever_mode=TreeRetrieverMode.SELECT_LEAF_EMBEDDING,
    child_branch_factor=3,
)

response = query_engine.query("介绍下颐和园的历史")
print(response.response)

颐和园的历史可以追溯到清朝时期。该园林最初名为清漪园，是乾隆皇帝为庆祝其母亲六十寿辰而建造的。1860年，英法联军入侵北京时，清漪园遭到严重破坏。1888年，慈禧太后重建了这座园林，并将其命名为颐和园，寓意“颐养天年”之意。

在重建过程中，颐和园融合了中国传统园林艺术与西方建筑风格，成为了一座集自然景观、人工建筑于一体的皇家园林。其中，长廊是颐和园的标志性建筑之一，全长728米，共有273间房屋，是中国古代建筑的杰作。

1900年，八国联军侵华战争期间，颐和园再次遭受破坏。1902年，清政府与各国达成协议，将颐和园归还给中国。此后，颐和园逐渐成为了一个对外开放的旅游景点。

2009年，颐和园被联合国教科文组织列入世界文化遗产名录，成为中国园林艺术的杰出代表之一。
CPU times: user 687 ms, sys: 3.58 ms, total: 691 ms
Wall time: 15.7 s
